In [1]:
# may need to import sys to fix libs pathing for this file
import sys
sys.path.append('../')
from libs.kraken_conn import kraken
import numpy as np
import pandas as pd
import datetime as dt

In [39]:
columns = {0:'datetime',1:'open',2:'high',3:'low',4:'close',5:'volume'}
def make_df(raw_df):
    df = raw_df.loc[:,:5]
    df.rename(columns=columns, inplace=True)
    df = df[["datetime", "close"]]
    
    df["timestamp"] = df["datetime"].apply(lambda x: dt.datetime.fromtimestamp(x/1000))
    df = df.drop(columns="datetime")
    df.set_index("timestamp", inplace=True)
    df.index=pd.to_datetime(df.index)
    return df

In [49]:
# fetchOHLCV will get ~last 30 days of hourly data
# bl = dt.datetime.fromtimestamp((k[0][0])/1000)
def get_historical(pair="BTC/USD"):
    k = kraken.fetchOHLCV(pair, "1h")
    
    df = pd.DataFrame()
    df = df.append(k)
    return make_df(df)

In [45]:
btc_curr = get_historical()
btc_curr.head()

,close
timestamp,
2021-05-31 19:00:00,37829.1
2021-05-31 20:00:00,37209.6
2021-05-31 21:00:00,36847.1
2021-05-31 22:00:00,36704.2
2021-05-31 23:00:00,36757.7


In [50]:
eth_curr = get_historical(pair="ETH/USD")
eth_curr.head()

,close
timestamp,
2021-05-31 19:00:00,2731.33
2021-05-31 20:00:00,2660.06
2021-05-31 21:00:00,2644.41
2021-05-31 22:00:00,2629.74
2021-05-31 23:00:00,2638.08


In [51]:
btc_curr.rename(columns={"close":"btc"}, inplace=True)
eth_curr.rename(columns={"close":"eth"}, inplace=True)

In [52]:
crypto_df = btc_curr.merge(eth_curr, how="inner", on="timestamp")

In [53]:
crypto_df.head()

,btc,eth
timestamp,,
2021-05-31 19:00:00,37829.1,2731.33
2021-05-31 20:00:00,37209.6,2660.06
2021-05-31 21:00:00,36847.1,2644.41
2021-05-31 22:00:00,36704.2,2629.74
2021-05-31 23:00:00,36757.7,2638.08


In [54]:
num_ticks = 60
symbols = ["btc","eth"]
shifted_df = pd.DataFrame()
for symbol in symbols:
    shifted_df[symbol] = crypto_df[symbol].copy()
    for i in range(1,num_ticks):
        name = f"{symbol}-{i}"
        shifted_df[name] = crypto_df[symbol].shift(i)
shifted_df.shape

(720, 120)

In [55]:
shifted_df.head()

,btc,btc-1,btc-2,btc-3,btc-4,btc-5,btc-6,btc-7,btc-8,btc-9,...,eth-50,eth-51,eth-52,eth-53,eth-54,eth-55,eth-56,eth-57,eth-58,eth-59
timestamp,,,,,,,,,,,,,,,,,,,,,
2021-05-31 19:00:00,37829.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-05-31 20:00:00,37209.6,37829.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-05-31 21:00:00,36847.1,37209.6,37829.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-05-31 22:00:00,36704.2,36847.1,37209.6,37829.1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-05-31 23:00:00,36757.7,36704.2,36847.1,37209.6,37829.1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
